# Goal:
Experiment with the hugging face library:
- download a simple model
- use it for inference
- extract the attention maps (perhaps via debugging)

In [14]:
from transformers import GPT2Tokenizer, GPTJModel, GPTJForCausalLM
import torch

tokenizer = GPT2Tokenizer.from_pretrained("ydshieh/tiny-random-gptj-for-causal-lm")
model = GPTJForCausalLM.from_pretrained("ydshieh/tiny-random-gptj-for-causal-lm")

#tokenizer = GPT2Tokenizer.from_pretrained("hf-internal-testing/tiny-random-gptj")
#model = GPTJModel.from_pretrained("hf-internal-testing/tiny-random-gptj")
#help(GPTJModel)

In [13]:
model

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(1000, 32)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTJBlock(
        (ln_1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=32, out_features=32, bias=False)
          (v_proj): Linear(in_features=32, out_features=32, bias=False)
          (q_proj): Linear(in_features=32, out_features=32, bias=False)
          (out_proj): Linear(in_features=32, out_features=32, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=32, out_features=128, bias=True)
          (fc_out): Linear(in_features=128, out_features=32, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPTJBlock(
        (ln_1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)

In [5]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
input_ids = inputs["input_ids"]
# return_tensors="pt" stands for pytorch as return type and not standard python
inputs

{'input_ids': tensor([[ 40, 416,  79,  12, 230,  89, 231, 432, 301, 212, 933]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [6]:
print("Ġ char represents the space." )
tokenizer.convert_ids_to_tokens(input_ids[0])  
# [0] is to get the first element in the batch (coincidentally also the only one)

Ġ char represents the space.


['H', 'ell', 'o', ',', 'Ġm', 'y', 'Ġd', 'og', 'Ġis', 'Ġc', 'ute']

In [8]:
help(tokenizer)

Help on GPT2Tokenizer in module transformers.models.gpt2.tokenization_gpt2 object:

class GPT2Tokenizer(transformers.tokenization_utils.PreTrainedTokenizer)
 |  GPT2Tokenizer(vocab_file, merges_file, errors='replace', unk_token='<|endoftext|>', bos_token='<|endoftext|>', eos_token='<|endoftext|>', add_prefix_space=False, **kwargs)
 |  
 |  Construct a GPT-2 tokenizer. Based on byte-level Byte-Pair-Encoding.
 |  
 |  This tokenizer has been trained to treat spaces like parts of the tokens (a bit like sentencepiece) so a word will
 |  be encoded differently whether it is at the beginning of the sentence (without space) or not:
 |  
 |  ::
 |  
 |      >>> from transformers import GPT2Tokenizer
 |      >>> tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
 |      >>> tokenizer("Hello world")['input_ids']
 |      [15496, 995]
 |      >>> tokenizer(" Hello world")['input_ids']
 |      [18435, 995]
 |  
 |  You can get around that behavior by passing ``add_prefix_space=True`` when instantiat

## Extract attention

In [7]:
outputs = model(**inputs, output_attentions=True)
# output_attentions=True is required, otherwise that field is not visible

# using output_hidden_states=True we can also get the 
# Hidden-states of the model at the output of each layer plus the initial embedding outputs.
outputs

CausalLMOutputWithPast(loss=None, logits=tensor([[[-0.0323,  0.0318, -0.0662,  ...,  0.1837,  0.0578, -0.1297],
         [-0.0123,  0.0069, -0.0111,  ...,  0.1814,  0.0552, -0.0187],
         [-0.0038,  0.0315, -0.0168,  ...,  0.0853,  0.0231, -0.2297],
         ...,
         [ 0.1834, -0.1816, -0.0232,  ..., -0.3520, -0.1645, -0.0654],
         [ 0.1003, -0.1515,  0.1125,  ..., -0.0605, -0.0996, -0.1787],
         [ 0.0786,  0.0333, -0.0046,  ..., -0.0942,  0.0644, -0.1715]]],
       grad_fn=<ViewBackward0>), past_key_values=((tensor([[[[-1.1634e-02,  1.6961e-02,  1.9648e-02,  3.7738e-02,  3.8144e-02,
           -1.9056e-01, -1.1523e-01, -1.3935e-01],
          [ 4.4637e-02,  1.2676e-01, -9.8951e-02,  1.5734e-02,  1.1940e-01,
            1.7108e-01,  4.0918e-02, -2.3823e-01],
          [-9.0130e-02, -1.9191e-02,  1.0203e-01, -2.2726e-01, -8.5284e-02,
           -6.4323e-02, -9.6822e-02,  5.4474e-02],
          [ 1.4120e-01, -9.3763e-02, -6.3809e-02,  2.1546e-01, -2.6702e-02,
         

In [8]:
attention_weights = outputs["attentions"]
# it returns a matrix n_tokens x n_tokens (because it is self-attention) x n_head
# for each layer in the model

# check model parameters (e.g. n_head = 4): 
# https://huggingface.co/hf-internal-testing/tiny-random-gptj/blob/main/config.json

print(f"The attention field contains {len(attention_weights)} items")  # 5 items, one for each layer
first_layer = attention_weights[0]
first_layer.shape

The attention field contains 5 items


torch.Size([1, 4, 11, 11])

In [10]:
# ONLY FOR GPTJModel
#last_hidden_states = outputs.last_hidden_state
#last_hidden_states  # Sequence of hidden-states at the output of the last layer of the model.

## Try to generate

In [11]:
greedy_output = model.generate(input_ids, max_length=50)
tokenizer.decode(greedy_output[0], skip_special_tokens=True)

'Hello, my dog is cute AfterJ����������������������������llHqere :keits wair'